![title](../images/header.png)

Merging CEO data phase II (2015-2022)
-------
This notebook merges validated data from different CEO CAFI projects for 2015-2022
###### For more information contact aurelie.shapiro@fao.org or remi.dannunzio@fao.org

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Set the display option to a large value to prevent text wrapping
pd.set_option('display.max_colwidth', None)

## Enter parameters

In [2]:
#identifier for country 
# COD COG CMR GAB EQG CAF
iso = 'EQG'

### 1. Read all CEO sample files
download your CEO sample files from collect.earth and upload to SEPAL using FileZilla.
add as many files as you have projects

In [3]:
#DRC
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/ceo-CAFI-DDD-RDC-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/ceo-CAFI-DDD-RDC-eSBAE-validation-2-sample-data.csv')
#ceo3 = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/ceo-CAFI-DDD-RDC-eSBAE-validation-3-sample-data.csv')
#GAB
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-2-sample-data.csv')
#ceo3 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-3-sample-data.csv')
#ceo4 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-4-sample-data.csv')
#ceo5 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-5-sample-data.csv')
#EQG
ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-Equatorial-Guinea-eSBAE-validation-1-sample-data.csv')
ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-Equatorial-Guinea-eSBAE-validation-2-sample-data.csv')
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-EQG-eSBAE-2023-collection-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-EQG-eSBAE-2023-collection-2-sample-data.csv')
#COG
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/COG/COG_all_ceo_1522_1622_clean.csv')
#CMR
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/CMR/ceo-CAFI-DDD-Cameroun-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/CMR/ceo-CAFI-DDD-Cameroun-eSBAE-validation-2-sample-data.csv')
#CAR
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/CAR/ceo-CAFI-DDD-RCA-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/CAR/ceo-CAFI-DDD-RCA-eSBAE-validation-2-sample-data.csv')

make a list of the ceo files for processing

In [4]:
#depends on how many files you have
# if you have 5 files
#ceo_files = [ceo1, ceo2, ceo3, ceo4, ceo5]
# if you have 3 files
#ceo_files = [ceo1, ceo2, ceo3]
# if you have 2 files
ceo_files = [ceo1,ceo2]
# if you have 1 file
#ceo_files = [ceo1]

In [5]:
# output file  CEO validated data file with select columns
ceo_data_out =  '/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_1522_clean.csv'

### end of parameters

In [6]:
column_names = ceo1.columns.tolist()
# Print the list of column names
print(column_names)

['plotid', 'sampleid', 'lon', 'lat', 'email', 'flagged', 'collection_time', 'analysis_duration', 'imagery_title', 'imagery_attributions', 'sample_geom', 'pl_aspect', 'pl_brightness_max', 'pl_red_mean', 'pl_greenness_max', 'pl_cusum_confidence', 'pl_cusum_change_date', 'pl_nir_max', 'pl_red_min', 'pl_bfast_means', 'pl_elevation', 'pl_red_sd', 'pl_cnc_1520', 'pl_swir1_sd', 'pl_index', 'pl_tmf_sub', 'pl_dw_tree_prob__min', 'pl_bs_slope_max', 'pl_red_max', 'pl_images', 'pl_dist', 'pl_bfast_magnitude', 'pl_ccdc_magnitude', 'pl_simple_combined', 'pl_tmf_defyear', 'pl_greenness_mean', 'pl_tmf_2019', 'pl_swir2_sd', 'pl_nir_sd', 'pl_gfc_tc00', 'pl_greenness_min', 'pl_tmf_degyear', 'pl_tmf_2020', 'pl_tmf_2017', 'pl_bs_slope_min', 'pl_lang_tree_height', 'pl_bs_slope_sd', 'pl_dw_class_mode', 'pl_swir1_mean', 'pl_wetness_sd', 'pl_bs_slope_mean', 'pl_ccdc_change_date', 'pl_dw_tree_prob_mean', 'pl_nir_mean', 'pl_dw_tree_prob__stddev', 'pl_tmf_2016', 'pl_nir_min', 'pl_esri_lc17', 'pl_wetness_mean', 'p

##### add a column named "interpreted" which = 1 when the point has been validated in CEO, otherwise 0
count the number of validated points per ceo file
replace forest and non-forest types with codes

In [7]:
# List to store the modified DataFrames
modified_ceo_files = []

# Initialize collection counter
collection_counter = 1

for i, ceo in enumerate(ceo_files):
       
    # Sort the DataFrame by 'date' in descending order
    ceo.sort_values(by='collection_time', ascending=True, inplace=True)

    # Remove duplicates keeping the last occurrence (the most recent date)
    ceo.drop_duplicates(subset='plotid', keep='first', inplace=True)

    # Reset the index
    ceo.reset_index(drop=True, inplace=True)
    
    # Add 'collection' column with an increasing number
    ceo['collection'] = collection_counter
    
    # Add 'interpreted' column with 1 when 'email' is not null, otherwise 0
    ceo['interpreted'] = np.where(ceo['email'].notna(), 1, 0)
    
    # Calculate total count for each 'interpreted' value
    interpreted_counts = ceo['interpreted'].value_counts().to_dict()
    
    # Append the modified DataFrame to the list
    modified_ceo_files.append(ceo)
    
    # Print count for the current DataFrame
    print(f"ceo {i + 1} # of validated points: {interpreted_counts.get(1, 0)}, # of not validated points: {interpreted_counts.get(0, 0)}")
    
    # Increment the collection counter
    collection_counter += 1

# Concatenate the modified DataFrames into one merged DataFrame
merged_ceo = pd.concat(modified_ceo_files, ignore_index=True)


# Specify the columns you want to keep in the merged DataFrame
columns_to_keep = ['plotid', 'pl_point_id','sampleid', 'lon', 'lat', 
                    'sample_geom', 'forêt ou non-forêt en 2015?', "Y'a t'il de la régéneration", 
                    'Changement entre 2015-2022', 'Type de Changement (2015-2022)', 'Exploitation forestière artisanale (<5ha)', 
                    'Activité minière artisanale', 'Infrastructure routière', 'Infrastructure d’urbanisation ou habitations', 
                    'Exploitation forestière industrielle (>5ha)', 'Autre moteur', 'Descriptif autre moteur', 'Activité minière industrielle', 
                    'Agriculture industrielle', 'Agriculture paysanne', 'Année de Changement', 'Type de non-forêt en 2015', 'Type de forêt en 2015', 'collection', 'interpreted']

# Select only the specified columns
merged_ceo = merged_ceo[columns_to_keep]

# Create a dictionary to map old column names to new column names
column_mapping = {
    'pl_point_id':'point_id',
    'forêt ou non-forêt en 2015?': 'Ref_FNF_2015', 
    "Y'a t'il de la régéneration": 'Ref_Regeneration',
    'Changement entre 2015-2022': 'Ref_Change_1522', 
    'Type de Changement (2015-2022)': 'Ref_Change_Type_1522', 
    'Exploitation forestière artisanale (<5ha)': 'ArtFor',
    'Activité minière artisanale': 'ArtMine',
    'Infrastructure routière': 'InfraR', 
    'Infrastructure d’urbanisation ou habitations': 'Urb',
    'Exploitation forestière industrielle (>5ha)':'IndFor', 
    'Autre moteur': 'Other', 
    'Descriptif autre moteur': 'Other_Desc', 
    'Activité minière industrielle': 'IndMine', 
    'Agriculture industrielle': 'IndAg', 
    'Agriculture paysanne': 'ArtAg', 
    'Année de Changement': 'Ref_Year_1522', 
    'Type de non-forêt en 2015':'Ref_NFtype_2015', 
    'Type de forêt en 2015':'Ref_Ftype_2015'
    #add other columns as needed
    #,'NOM': 'Province' # DRC province
}

# Use the rename() method to rename the columns
merged_ceo = merged_ceo.rename(columns=column_mapping)

# Define the replacements as a dictionary
replace_dict = {
    'Ref_FNF_2015': {'non-forêt': '0', 'forêt': '1'},
    'Ref_Ftype_2015': {'1 - forêt dense': '1', '2 - forêt dense sèche': '2', '3 - forêt secondaire': '3','4 - forêt claire sèche': '4','7 - forêt mangrove': '7',
                 '8 - forêt marécageuse': '8', '9 - forêt galérie': '9','10 - plantation forestière': '10', '11- savane arborée': '11'},
    'Ref_NFtype_2015': {'12 - savane arbustive': '12', '13 - savane herbacée': '13', '14 - prairie aquatique': '14', '15 - sol nu végétation éparse': '15',
                  '16 - terres cultivées': '16', '17 - zone baties': '17', '18 - eau': '18'},
    'Ref_Change_1522': {'oui':'1','non':'0','':'0'},
    'ArtFor':{'oui':'1','non':'0','':'0'},
    'IndFor':{'oui':'1','non':'0','':'0'},
    'ArtAg':{'oui':'1','non':'0','':'0'},
    'IndAg':{'oui':'1','non':'0','':'0'},
    'ArtMine':{'oui':'1','non':'0','':'0'},
    'IndMine':{'oui':'1','non':'0','':'0'},
    'Urb':{'oui':'1','non':'0','':'0'},
    'InfraR':{'oui':'1','non':'0','':'0'},
    'Other':{'oui':'1','non':'0','':'0'}
}

# Use the replace() method to replace strings in specified columns
merged_ceo.replace(replace_dict, inplace=True)


columns_to_fill = ['Ref_Change_1522','ArtAg','ArtFor','IndFor','IndAg','ArtMine','IndMine','Urb','InfraR','Other']
# Set NaN values in the selected columns to 0
merged_ceo[columns_to_fill] = merged_ceo[columns_to_fill].fillna('0')
merged_ceo['collection'] = 'coll_1522_' + merged_ceo['collection'].astype(str)

ceo 1 # of validated points: 498, # of not validated points: 0
ceo 2 # of validated points: 3984, # of not validated points: 0


remove non validated points and remove duplicates

In [8]:
merged_ceo_interpreted = merged_ceo[merged_ceo['interpreted'] == 1]

In [9]:
ceo_column_names = merged_ceo_interpreted.columns.tolist()
# Print the list of column names
print(ceo_column_names)

['plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted']


In [10]:
merged_ceo_interpreted.head()

,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,Ref_Change_1522,Ref_Change_Type_1522,ArtFor,ArtMine,InfraR,Urb,IndFor,Other,Other_Desc,IndMine,IndAg,ArtAg,Ref_Year_1522,Ref_NFtype_2015,Ref_Ftype_2015,collection,interpreted
0,45,45,45,8.460650,3.453988,POINT(8.46065 3.453988),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1
1,65,65,65,8.431363,3.357773,POINT(8.431363 3.357773),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1
2,117,117,117,8.441585,3.319245,POINT(8.441585 3.319245),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1
3,108,108,108,8.490696,3.434682,POINT(8.490696 3.434682),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1
4,122,122,122,8.466201,3.367339,POINT(8.466201 3.367339),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1


In [11]:
merged_ceo_interpreted = merged_ceo_interpreted.sort_values(by='collection', ascending=True)

# Remove duplicates keeping the last occurrence (the most recent date)
merged_ceo_nodup = merged_ceo_interpreted.drop_duplicates(subset='plotid', keep='first')

# Reset the index
merged_ceo_nodup = merged_ceo_nodup.reset_index(drop=True)

##### count points in different categories

In [12]:
merged_ceo_nodup['interpreted'].value_counts(dropna=False)

interpreted
1    4482
Name: count, dtype: int64

In [13]:
pd.pivot_table(merged_ceo_nodup,values='plotid',index=['Ref_Change_1522'],columns=['interpreted'],aggfunc="count")

interpreted,1
Ref_Change_1522,
0,4156
1,326


In [14]:
merged_ceo_nodup['Ref_FNF_2015'].value_counts(dropna=False)

Ref_FNF_2015
1    3604
0     878
Name: count, dtype: int64

In [15]:
merged_ceo_nodup['Ref_Change_Type_1522'].value_counts(dropna=False)

Ref_Change_Type_1522
NaN              4156
déforestation     193
dégradation       133
Name: count, dtype: int64

In [16]:
merged_ceo_nodup['collection'].value_counts(dropna=False)

collection
coll_1522_2    3984
coll_1522_1     498
Name: count, dtype: int64

##### create a LC type column

In [17]:
merged_ceo_nodup['Ref_LCover_2015'] = merged_ceo_nodup['Ref_Ftype_2015'].fillna(merged_ceo_nodup['Ref_NFtype_2015'])
merged_ceo_nodup['Ref_LCover_2015'].value_counts(dropna=False)

Ref_LCover_2015
1     2125
3      856
2      413
15     356
16     238
17     166
7       56
9       51
18      47
14      44
10      35
4       30
8       27
13      18
11      11
12       9
Name: count, dtype: int64

##### Check if values in 'pointid' are unique

In [18]:
# this might take some time
unique_check = merged_ceo_nodup['plotid'].nunique() == len(merged_ceo_nodup['plotid'])

# Print the result
if unique_check:
    print("All values are unique!")
else:
    print("F$%§!")

All values are unique!


##### if there are duplicates determine which points and which project (collection) and fix these in CEO: find the point id and enter the same correct interpretation for both users

In [19]:
# Find duplicate values in the 'ID' column
#duplicate_ids = merged_ceo_nodup[merged_ceo_nodup.duplicated(subset='point_id', keep='first')]

# If you want to print specific columns along with duplicates, you can do:
# For example, print only 'ID' and 'Name' columns
#print("\nDuplicate Rows with 'ID' and 'Name':")
#print(duplicate_ids[['point_id', 'collection']])
#len(duplicate_ids)

In [20]:
#merged_ceo_nodup.head()

### 2. add new columns for change and year

clean up columns and replace values

In [21]:
def f(x):
  if x['Ref_FNF_2015'] == '0' : return 'NF'
  elif x['Ref_Change_Type_1522'] == 'déforestation': return 'Def'
  elif x['Ref_Change_Type_1522'] == 'dégradation' : return 'Deg'
  else: return 'Stable'

merged_ceo_nodup['Ref_Change_Type_1522'] = merged_ceo_nodup.apply(f, axis=1)
merged_ceo_nodup['Ref_Change_Type_1522'].value_counts()

Ref_Change_Type_1522
Stable    3278
NF         878
Def        193
Deg        133
Name: count, dtype: int64

In [22]:
def y(x):
  if x['Ref_Change_Type_1522'] == 'Def' and x['Ref_Year_1522'] == 2016: return 'Def2016'
  elif x['Ref_Change_Type_1522'] == 'Def' and x['Ref_Year_1522'] == 2017: return 'Def2017'
  elif x['Ref_Change_Type_1522'] == 'Def' and x['Ref_Year_1522'] == 2018: return 'Def2018'
  elif x['Ref_Change_Type_1522'] == 'Def' and x['Ref_Year_1522'] == 2019: return 'Def2019'
  elif x['Ref_Change_Type_1522'] == 'Def' and x['Ref_Year_1522'] == 2020: return 'Def2020'
  elif x['Ref_Change_Type_1522'] == 'Def' and x['Ref_Year_1522'] == 2021: return 'Def2021'
  elif x['Ref_Change_Type_1522'] == 'Def' and x['Ref_Year_1522'] == 2022: return 'Def2022'
  elif x['Ref_Change_Type_1522'] == 'Deg' and x['Ref_Year_1522'] == 2016: return 'Deg2016'
  elif x['Ref_Change_Type_1522'] == 'Deg' and x['Ref_Year_1522'] == 2017: return 'Deg2017'
  elif x['Ref_Change_Type_1522'] == 'Deg' and x['Ref_Year_1522'] == 2018: return 'Deg2018'
  elif x['Ref_Change_Type_1522'] == 'Deg' and x['Ref_Year_1522'] == 2019: return 'Deg2019'
  elif x['Ref_Change_Type_1522'] == 'Deg' and x['Ref_Year_1522'] == 2020: return 'Deg2020'
  elif x['Ref_Change_Type_1522'] == 'Deg' and x['Ref_Year_1522'] == 2021: return 'Deg2021'
  elif x['Ref_Change_Type_1522'] == 'Deg' and x['Ref_Year_1522'] == 2022: return 'Deg2022'
  elif x['Ref_Change_Type_1522'] == 'NF' : return 'NF'
  elif x['Ref_Change_Type_1522'] == 'Stable' : return 'Stable'
  else: return ''

merged_ceo_nodup['Ref_Change_Year_1522'] = merged_ceo_nodup.apply(y, axis=1)

annual_counts = merged_ceo_nodup['Ref_Change_Year_1522'].value_counts()
annual_counts = annual_counts.sort_index()
print(annual_counts)

Ref_Change_Year_1522
Def2016      38
Def2017      42
Def2018      32
Def2019      25
Def2020      25
Def2021      19
Def2022      12
Deg2016       9
Deg2017      31
Deg2018      38
Deg2019      15
Deg2020       6
Deg2021      17
Deg2022      17
NF          878
Stable     3278
Name: count, dtype: int64


In [23]:
# Define the new column based on conditions
merged_ceo_nodup['Ref_Change_LCover_1522'] = merged_ceo_nodup.apply(lambda row: f"{row['Ref_Change_Type_1522']}_{row['Ref_LCover_2015']}", axis=1)

In [24]:
annual_counts.sum()

4482

In [25]:
pd.pivot_table(merged_ceo_nodup,values='point_id',index=['Ref_Change_LCover_1522'],columns=['interpreted'],aggfunc="count",margins=True,
                             margins_name='Total')

interpreted,1,Total
Ref_Change_LCover_1522,,
Def_1,47,47
Def_10,2,2
Def_2,10,10
Def_3,133,133
Def_4,1,1
Deg_1,34,34
Deg_10,9,9
Deg_2,3,3
Deg_3,83,83


In [26]:
pd.pivot_table(merged_ceo_nodup,values='point_id',index=['Ref_Change_Year_1522'],columns=['interpreted'],aggfunc="count",margins=True,
                             margins_name='Total')

interpreted,1,Total
Ref_Change_Year_1522,,
Def2016,38,38
Def2017,42,42
Def2018,32,32
Def2019,25,25
Def2020,25,25
Def2021,19,19
Def2022,12,12
Deg2016,9,9
Deg2017,31,31


In [27]:
pd.pivot_table(merged_ceo_nodup,values='point_id',index=['Ref_Change_Type_1522'],columns=['interpreted'],aggfunc="count",margins=True,
                             margins_name='Total')

interpreted,1,Total
Ref_Change_Type_1522,,
Def,193,193
Deg,133,133
NF,878,878
Stable,3278,3278
Total,4482,4482


In [28]:
merged_ceo_nodup['Def2016'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Def2016', 1, 0)
merged_ceo_nodup['Def2017'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Def2017', 1, 0)
merged_ceo_nodup['Def2018'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Def2018', 1, 0)
merged_ceo_nodup['Def2019'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Def2019', 1, 0)
merged_ceo_nodup['Def2020'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Def2020', 1, 0)
merged_ceo_nodup['Def2021'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Def2021', 1, 0)
merged_ceo_nodup['Def2022'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Def2022', 1, 0)

In [29]:
merged_ceo_nodup['Deg2016'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Deg2016', 1, 0)
merged_ceo_nodup['Deg2017'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Deg2017', 1, 0)
merged_ceo_nodup['Deg2018'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Deg2018', 1, 0)
merged_ceo_nodup['Deg2019'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Deg2019', 1, 0)
merged_ceo_nodup['Deg2020'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Deg2020', 1, 0)
merged_ceo_nodup['Deg2021'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Deg2021', 1, 0)
merged_ceo_nodup['Deg2022'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Deg2022', 1, 0)

In [30]:
merged_ceo_nodup['Defall'] = np.where(merged_ceo_nodup['Ref_Change_Type_1522'] == 'Def', 1, 0)
merged_ceo_nodup['Degall'] = np.where(merged_ceo_nodup['Ref_Change_Type_1522'] == 'Deg', 1, 0)

In [31]:
merged_ceo_nodup['Stable'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'Stable', 1, 0)
merged_ceo_nodup['NF'] = np.where(merged_ceo_nodup['Ref_Change_Year_1522'] == 'NF', 1, 0)

In [32]:
merged_ceo_nodup['DensFor'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '1', 1, 0).astype(int)
merged_ceo_nodup['DensDryFor'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '2', 1, 0).astype(int)
merged_ceo_nodup['SecFor'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '3', 1, 0).astype(int)
merged_ceo_nodup['DryOpenFor'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '4', 1, 0).astype(int)
#merged_ceo_nodup['SubMont'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '5', 1, 0).astype(int)
#merged_ceo_nodup['MontFor'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '6', 1, 0).astype(int)
merged_ceo_nodup['Mangrove'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '7', 1, 0).astype(int)
merged_ceo_nodup['Swamp'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '8', 1, 0).astype(int)
merged_ceo_nodup['Gallery'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '9', 1, 0).astype(int)
merged_ceo_nodup['Plantation'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '10', 1, 0).astype(int)
merged_ceo_nodup['Woodland'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '11', 1, 0).astype(int)
merged_ceo_nodup['Shrubland'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '12', 1, 0).astype(int)
merged_ceo_nodup['Grassland'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '13', 1, 0).astype(int)
merged_ceo_nodup['Aquatic'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '14', 1, 0).astype(int)
merged_ceo_nodup['Bare'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '15', 1, 0).astype(int)
merged_ceo_nodup['Cultivated'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '16', 1, 0).astype(int)
merged_ceo_nodup['Builtup'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '17', 1, 0).astype(int)
merged_ceo_nodup['Water'] = np.where(merged_ceo_nodup['Ref_LCover_2015'] == '18', 1, 0).astype(int)

In [33]:
merged_ceo_nodup['DensFor_Def'] = ((merged_ceo_nodup['DensFor'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['DensDryFor_Def'] = ((merged_ceo_nodup['DensDryFor'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['SecFor_Def'] = ((merged_ceo_nodup['SecFor'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['DryOpenFor_Def'] = ((merged_ceo_nodup['DryOpenFor'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['Mangrove_Def'] = ((merged_ceo_nodup['Mangrove'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['Swamp_Def'] = ((merged_ceo_nodup['Swamp'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['Gallery_Def'] = ((merged_ceo_nodup['Gallery'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['Plantation_Def'] = ((merged_ceo_nodup['Plantation'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)
merged_ceo_nodup['Woodland_Def'] = ((merged_ceo_nodup['Woodland'] == 1) & (merged_ceo_nodup['Defall'] == 1)).astype(int)

In [34]:
merged_ceo_nodup['DensFor_Deg'] = ((merged_ceo_nodup['DensFor'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['DensDryFor_Deg'] = ((merged_ceo_nodup['DensDryFor'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['SecFor_Deg'] = ((merged_ceo_nodup['SecFor'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['DryOpenFor_Deg'] = ((merged_ceo_nodup['DryOpenFor'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['Mangrove_Deg'] = ((merged_ceo_nodup['Mangrove'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['Swamp_Deg'] = ((merged_ceo_nodup['Swamp'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['Gallery_Deg'] = ((merged_ceo_nodup['Gallery'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['Plantation_Deg'] = ((merged_ceo_nodup['Plantation'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)
merged_ceo_nodup['Woodland_Deg'] = ((merged_ceo_nodup['Woodland'] == 1) & (merged_ceo_nodup['Degall'] == 1)).astype(int)

In [35]:
merged_ceo_nodup['DensFor_stable'] = ((merged_ceo_nodup['DensFor'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['DensDryFor_stable'] = ((merged_ceo_nodup['DensDryFor'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['SecFor_stable'] = ((merged_ceo_nodup['SecFor'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['DryOpenFor_stable'] = ((merged_ceo_nodup['DryOpenFor'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['Mangrove_stable'] = ((merged_ceo_nodup['Mangrove'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['Mangrove_stable'] = ((merged_ceo_nodup['Swamp'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['Mangrove_stable'] = ((merged_ceo_nodup['Gallery'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['Plantation_stable'] = ((merged_ceo_nodup['Plantation'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)
merged_ceo_nodup['Woodland_stable'] = ((merged_ceo_nodup['Woodland'] == 1) & (merged_ceo_nodup['Stable'] == 1)).astype(int)

In [36]:
print(merged_ceo_nodup.dtypes)

plotid                      int64
point_id                    int64
sampleid                    int64
lon                       float64
lat                       float64
sample_geom                object
Ref_FNF_2015               object
Ref_Regeneration           object
Ref_Change_1522            object
Ref_Change_Type_1522       object
ArtFor                     object
ArtMine                    object
InfraR                     object
Urb                        object
IndFor                     object
Other                      object
Other_Desc                 object
IndMine                    object
IndAg                      object
ArtAg                      object
Ref_Year_1522             float64
Ref_NFtype_2015            object
Ref_Ftype_2015             object
collection                 object
interpreted                 int64
Ref_LCover_2015            object
Ref_Change_Year_1522       object
Ref_Change_LCover_1522     object
Def2016                     int64
Def2017       

In [37]:
columns_to_convert = ['Def2016','Def2017','Def2018','Def2019','Def2020','Def2021','Def2022','Deg2016','Deg2017','Deg2018','Deg2019','Deg2020','Deg2021','Deg2022','NF',
'DensFor','DensDryFor','SecFor','DryOpenFor','Mangrove','Swamp','Gallery','Plantation','Woodland','Shrubland','Grassland','Aquatic','Bare','Cultivated','Builtup','Water',
'DensFor_Def','DensDryFor_Def','SecFor_Def','DryOpenFor_Def','Mangrove_Def','Swamp_Def','Gallery_Def','Plantation_Def','Woodland_Def',
'DensFor_Deg','DensDryFor_Deg','SecFor_Deg','DryOpenFor_Deg','Mangrove_Deg','Swamp_Deg','Gallery_Deg','Plantation_Deg','Woodland_Deg',
'DensFor_stable','DensDryFor_stable','SecFor_stable','DryOpenFor_stable','Mangrove_stable','Plantation_stable','Woodland_stable']
merged_ceo_nodup[columns_to_convert] = merged_ceo_nodup[columns_to_convert].fillna(0)

In [38]:
pd.pivot_table(merged_ceo_nodup,values='point_id',index=['DensFor_Def'],columns=['interpreted'],aggfunc="count",margins=True,
                             margins_name='Total')

interpreted,1,Total
DensFor_Def,,
0,4435,4435
1,47,47
Total,4482,4482


In [39]:
merged_ceo_column_names = merged_ceo_nodup.columns.tolist()
# Print the list of column names
print(merged_ceo_column_names)

['plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrove_Deg', 'Swam

In [40]:
merged_ceo_nodup.head()

,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,Ref_Change_1522,Ref_Change_Type_1522,ArtFor,ArtMine,InfraR,Urb,IndFor,Other,Other_Desc,IndMine,IndAg,ArtAg,Ref_Year_1522,Ref_NFtype_2015,Ref_Ftype_2015,collection,interpreted,Ref_LCover_2015,Ref_Change_Year_1522,Ref_Change_LCover_1522,Def2016,Def2017,Def2018,Def2019,Def2020,Def2021,Def2022,Deg2016,Deg2017,Deg2018,Deg2019,Deg2020,Deg2021,Deg2022,Defall,Degall,Stable,NF,DensFor,DensDryFor,SecFor,DryOpenFor,Mangrove,Swamp,Gallery,Plantation,Woodland,Shrubland,Grassland,Aquatic,Bare,Cultivated,Builtup,Water,DensFor_Def,DensDryFor_Def,SecFor_Def,DryOpenFor_Def,Mangrove_Def,Swamp_Def,Gallery_Def,Plantation_Def,Woodland_Def,DensFor_Deg,DensDryFor_Deg,SecFor_Deg,DryOpenFor_Deg,Mangrove_Deg,Swamp_Deg,Gallery_Deg,Plantation_Deg,Woodland_Deg,DensFor_stable,DensDryFor_stable,SecFor_stable,DryOpenFor_stable,Mangrove_stable,Plantation_stable,Woodland_stable
0,45,45,45,8.460650,3.453988,POINT(8.46065 3.453988),1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1,1,Stable,Stable_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,15952,15952,15952,10.827311,1.756878,POINT(10.827311 1.756878),0,non,0,NF,0,0,0,0,0,0,NaN,0,0,0,NaN,15,NaN,coll_1522_1,1,15,NF,NF_15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,15936,15936,15936,10.747901,1.602624,POINT(10.747901 1.602624),1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1,1,Stable,Stable_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,15893,15893,15893,10.534713,1.187889,POINT(10.534713 1.187889),1,NaN,1,Deg,1,0,1,0,1,0,NaN,0,0,0,2022.0,NaN,1,coll_1522_1,1,1,Deg2022,Deg_1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15847,15847,15847,10.906755,1.930372,POINT(10.906755 1.930372),1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1,1,Stable,Stable_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [41]:
print(merged_ceo_nodup.columns.values)

['plotid' 'point_id' 'sampleid' 'lon' 'lat' 'sample_geom' 'Ref_FNF_2015'
 'Ref_Regeneration' 'Ref_Change_1522' 'Ref_Change_Type_1522' 'ArtFor'
 'ArtMine' 'InfraR' 'Urb' 'IndFor' 'Other' 'Other_Desc' 'IndMine' 'IndAg'
 'ArtAg' 'Ref_Year_1522' 'Ref_NFtype_2015' 'Ref_Ftype_2015' 'collection'
 'interpreted' 'Ref_LCover_2015' 'Ref_Change_Year_1522'
 'Ref_Change_LCover_1522' 'Def2016' 'Def2017' 'Def2018' 'Def2019'
 'Def2020' 'Def2021' 'Def2022' 'Deg2016' 'Deg2017' 'Deg2018' 'Deg2019'
 'Deg2020' 'Deg2021' 'Deg2022' 'Defall' 'Degall' 'Stable' 'NF' 'DensFor'
 'DensDryFor' 'SecFor' 'DryOpenFor' 'Mangrove' 'Swamp' 'Gallery'
 'Plantation' 'Woodland' 'Shrubland' 'Grassland' 'Aquatic' 'Bare'
 'Cultivated' 'Builtup' 'Water' 'DensFor_Def' 'DensDryFor_Def'
 'SecFor_Def' 'DryOpenFor_Def' 'Mangrove_Def' 'Swamp_Def' 'Gallery_Def'
 'Plantation_Def' 'Woodland_Def' 'DensFor_Deg' 'DensDryFor_Deg'
 'SecFor_Deg' 'DryOpenFor_Deg' 'Mangrove_Deg' 'Swamp_Deg' 'Gallery_Deg'
 'Plantation_Deg' 'Woodland_Deg' 'DensFor_

In [42]:
merged_ceo_nodup['Ref_Change_Year_1522'].value_counts(dropna=False)

Ref_Change_Year_1522
Stable     3278
NF          878
Def2017      42
Deg2018      38
Def2016      38
Def2018      32
Deg2017      31
Def2020      25
Def2019      25
Def2021      19
Deg2021      17
Deg2022      17
Deg2019      15
Def2022      12
Deg2016       9
Deg2020       6
Name: count, dtype: int64

In [43]:
merged_ceo_nodup.head()

,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,Ref_Change_1522,Ref_Change_Type_1522,ArtFor,ArtMine,InfraR,Urb,IndFor,Other,Other_Desc,IndMine,IndAg,ArtAg,Ref_Year_1522,Ref_NFtype_2015,Ref_Ftype_2015,collection,interpreted,Ref_LCover_2015,Ref_Change_Year_1522,Ref_Change_LCover_1522,Def2016,Def2017,Def2018,Def2019,Def2020,Def2021,Def2022,Deg2016,Deg2017,Deg2018,Deg2019,Deg2020,Deg2021,Deg2022,Defall,Degall,Stable,NF,DensFor,DensDryFor,SecFor,DryOpenFor,Mangrove,Swamp,Gallery,Plantation,Woodland,Shrubland,Grassland,Aquatic,Bare,Cultivated,Builtup,Water,DensFor_Def,DensDryFor_Def,SecFor_Def,DryOpenFor_Def,Mangrove_Def,Swamp_Def,Gallery_Def,Plantation_Def,Woodland_Def,DensFor_Deg,DensDryFor_Deg,SecFor_Deg,DryOpenFor_Deg,Mangrove_Deg,Swamp_Deg,Gallery_Deg,Plantation_Deg,Woodland_Deg,DensFor_stable,DensDryFor_stable,SecFor_stable,DryOpenFor_stable,Mangrove_stable,Plantation_stable,Woodland_stable
0,45,45,45,8.460650,3.453988,POINT(8.46065 3.453988),1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1,1,Stable,Stable_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,15952,15952,15952,10.827311,1.756878,POINT(10.827311 1.756878),0,non,0,NF,0,0,0,0,0,0,NaN,0,0,0,NaN,15,NaN,coll_1522_1,1,15,NF,NF_15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,15936,15936,15936,10.747901,1.602624,POINT(10.747901 1.602624),1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1,1,Stable,Stable_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,15893,15893,15893,10.534713,1.187889,POINT(10.534713 1.187889),1,NaN,1,Deg,1,0,1,0,1,0,NaN,0,0,0,2022.0,NaN,1,coll_1522_1,1,1,Deg2022,Deg_1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15847,15847,15847,10.906755,1.930372,POINT(10.906755 1.930372),1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_1522_1,1,1,Stable,Stable_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [44]:
merged_ceo_nodup['ISO'] = iso

### 3. plot change points by year

In [ ]:
#select the values you want to plot
values_to_plot = ['Def2016','Def2017','Def2018','Def2019','Def2020','Def2021','Def2022']

# Filter the DataFrame to include only the selected values
filtered_df = merged_ceo_nodup[merged_ceo_nodup['Ref_Change_Year_1522'].isin(values_to_plot)]

# Get the value counts and sort by index (alphabetically)
defvalue_counts = filtered_df['Ref_Change_Year_1522'].value_counts().sort_index()

# Plot the value counts as a bar graph
plt.figure(figsize=(8, 6))  # Optional: Set the figure size
defvalue_counts.plot(kind='bar', color='skyblue')


# Set plot labels and title
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('# of def points per year')

# Show the plot
plt.show()

##### plot degradation points by year

In [ ]:
#plot DEG 
#select the values you want to plot
values_to_plot = ['Deg2016','Deg2017','Deg2018','Deg2019','Deg2020','Deg2021','Deg2022']

# Filter the DataFrame to include only the selected values
filtered_df = merged_ceo_nodup[merged_ceo_nodup['Ref_Change_Year_1522'].isin(values_to_plot)]

# Get the value counts and sort by index (alphabetically)
degvalue_counts = filtered_df['Ref_Change_Year_1522'].value_counts().sort_index()

# Plot the value counts as a bar graph
plt.figure(figsize=(8, 6))  # Optional: Set the figure size
degvalue_counts.plot(kind='bar', color='skyblue')


# Set plot labels and title
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('# of deg points per year')

# Show the plot
plt.show()

In [ ]:
print(merged_ceo_nodup.dtypes)

### 4. export ceo data to one csv file

In [ ]:
merged_ceo_nodup.to_csv(ceo_data_out,index_label='ID')

###### you can run script 5D to merge these data to 2015-2020 strata for area estimation